In [ ]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.image as mpimg

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from keras.datasets import mnist

import visualkeras

In [29]:
# loading dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X = np.concatenate([X_train, X_test], axis = 0)
y = np.concatenate([y_train, y_test], axis = 0)

# reshapring 
X = X.reshape(X.shape[0], 28, 28, 1) 

# casting to float 
X.astype('float32')

# normalizing 
X = X / 255

# encoding output values
y = to_categorical(y)

In [63]:
INPUT_SHAPE = (28,28,1)
OUTPUT_SHAPE = 10
BATCH_SIZE = 6
EPOCHS = 2
VERBOSE = 2
K_FOLDS = 1

In [64]:
def create_model():
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=INPUT_SHAPE))
    model.add(MaxPool2D((2,2)))

    model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    model.add(MaxPool2D((2,2)))

    model.add(Flatten())

    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(10, activation='softmax'))
    
    model.compile(
        optimizer='adam', 
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )
    
    return model

In [65]:
model = create_model()
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 13, 13, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 5, 5, 64)         0         
 g2D)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 1600)              0         
                                                                 
 dense_15 (Dense)            (None, 128)              

In [ ]:
scores = []
models = []

for i in range(K_FOLDS):
    print("Training on Fold: ",i+1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3
                                               ,random_state = np.random.randint(1,1000, 1)[0])
    
    model = create_model()
    model.fit(
        X_train, 
        y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=VERBOSE,
        validation_split=0.3
    )
    
    scores.append(
        model.evaluate(X_test, y_test, verbose = 0)
    )
    models.append(model)
   
    print("======="*12, end="\n\n\n")

Training on Fold:  1
Epoch 1/2
5717/5717 - 27s - loss: 0.1902 - accuracy: 0.9424 - val_loss: 0.0697 - val_accuracy: 0.9788 - 27s/epoch - 5ms/step
Epoch 2/2


In [ ]:
print(f"K{K_FOLDS} Fold cross results:")
print("Accuracy:", np.mean(scores) * 100)
print("Std:",      np.std(scores) * 100)
print("K value:", len(scores))

In [ ]:
visualkeras.layered_view(models[0], legend=True, draw_volume=False,spacing=30)